## 임베딩 읽어들이기

In [10]:
def load_word_embeddings(path, num_vecs=30000):
    embeddings, vocab = [], []
    f = open(path, "r", encoding="utf-8").readlines()[1:num_vecs + 1]
    for line in f:
        splited_line = line.strip().split(" ")
        word = splited_line[0]
        vec = [float(el) for el in splited_line[1:]]
        vocab.append(word)
        embeddings.append(vec)
    word_to_index = {word: idx for idx, word in enumerate(vocab)}
    index_to_word = {idx: word for idx, word in enumerate(vocab)}
    vocab_size = len(vocab)
    return embeddings, vocab, word_to_index, index_to_word, vocab_size

In [11]:
embeddings, vocab, word_to_index, index_to_word, vocab_size = load_word_embeddings("embedding/word2vec.txt")

In [12]:
len(embeddings)

23599

In [9]:
word_to_index['폰']

43

## 관심 기능 정의

In [13]:
functions = ["배터리", "촬영", "디자인", "화면", "OS", "색"] # 배터리 스코어, 촬영 스코어, 디자인 스코어 .... 
function_size = len(functions)

## 거리 행렬 구하기

In [14]:
import numpy as np
from scipy.spatial import distance

distance_matrix = np.zeros([function_size, vocab_size], dtype=np.float16) # 0 으로 초기화 , 관심 기능 6 by 행렬의 차원(vocab_score: 2만3천개 정도)

In [16]:
for func_idx, func in enumerate(functions):
    func_vec = embeddings[word_to_index[func]]
    for word_idx, word_vec in enumerate(embeddings):
        distance_matrix[func_idx, word_idx] = distance.euclidean(func_vec, word_vec)

In [18]:
distance_matrix # 배터리, 촬영, 디자인, 화면, os, 색

array([[3.936, 4.203, 3.814, ..., 3.205, 3.22 , 3.238],
       [4.27 , 4.547, 4.35 , ..., 3.537, 3.648, 3.549],
       [3.623, 3.998, 3.59 , ..., 2.908, 3.037, 2.994],
       [3.945, 4.344, 3.97 , ..., 3.363, 3.523, 3.408],
       [3.863, 4.2  , 3.977, ..., 3.184, 3.293, 3.19 ],
       [4.22 , 4.492, 4.223, ..., 3.504, 3.54 , 3.578]], dtype=float16)

## 가중치 행렬 구하기

In [20]:
# 벡터 간 유클리디언 거리를 정규 분포로 바꾸는 식 
weight_matrix = np.zeros([function_size, vocab_size], dtype=np.float16)
for func_idx, per_func_distances in enumerate(distance_matrix):
    values = np.exp(-(per_func_distances ** 2) / 15)
    mean = np.mean(values)
    weight_matrix[func_idx, :] = values - mean + 0.5 # 평균을 빼고 0.5를 더한다 (0.5를 중심으로 0-1 사이 스코어로 정규 분포화 한다)

In [21]:
weight_matrix

array([[0.4243, 0.3762, 0.4475, ..., 0.5723, 0.569 , 0.5654],
       [0.4324, 0.3882, 0.419 , ..., 0.5703, 0.548 , 0.568 ],
       [0.4246, 0.3523, 0.4312, ..., 0.5767, 0.5483, 0.5576],
       [0.4507, 0.3801, 0.4458, ..., 0.5664, 0.533 , 0.557 ],
       [0.4397, 0.3784, 0.4185, ..., 0.5786, 0.5547, 0.577 ],
       [0.4197, 0.3748, 0.4192, ..., 0.5557, 0.5483, 0.5405]],
      dtype=float16)

## 가중치 행렬 시각화

In [22]:
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show

output_notebook()
def make_plot(title, hist, edges):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.y_range.start = 0
    p.grid.grid_line_color="white"
    return p

Loading BokehJS ...

In [23]:
plots = []
for func_idx, func in enumerate(functions):
    hist, edges = np.histogram(weight_matrix[func_idx], density=True, bins=50)
    plots.append(make_plot(func, hist, edges))
show(gridplot(plots, ncols=2, plot_width=400, plot_height=300, toolbar_location=None))
# 1에 가까운 값이 많이 없음 (가까운 거리 값이 너무 작아서 없는 것처럼 보임)

In [24]:
battery_weights = [(token, score) for token, score in zip(vocab, weight_matrix[0])]
#처음에 관심 기능에 battery를 정의 해 놨기 때문에, 첫번째 행 벡터임. 그걸 zip한거 

In [47]:
mean

0.3855

In [48]:
np.exp(0) - mean + 0.5 # 0.5는 임의로 준 값 (그냥 0.5 주변에 몰려있는 정규 분포를 만들겠다 )
# np.exp(0) - mean : 표준 정규 분포화 만들기 

1.114501953125

In [25]:
sorted(battery_weights, key=lambda x: x[1], reverse=True)
# 스코어가 가장 높은 건 자기 자신 (배터리 - 1), 그럴 듯해 보임.
# 스코어 상위 단어에 해당 문장이 포함되어 있으면 배터리 스코어가 높게 나와서 상위에 잡힘 
# 위에서 우리가 표준 정규분포화를 만들어 0.5를 더해서 그 주변에 몰려있는 정규 분포를 만들었으니까, 배터리 다음 밧데리 그리고 터리가 1값이 넘지 않고
# 0.7정도로 바로 떨어짐 

[('배터리', 1.068),
 ('밧데리', 0.9473),
 ('터리', 0.7573),
 ('표준형', 0.736),
 ('대용량', 0.7246),
 ('러닝', 0.704),
 ('리튬', 0.696),
 ('충전기', 0.6865),
 ('폴리머', 0.678),
 ('발열', 0.6724),
 ('대기전력', 0.6675),
 ('TTA', 0.662),
 ('간당간당', 0.659),
 ('급속', 0.6567),
 ('슬림형', 0.6567),
 ('커넥터', 0.652),
 ('뒷부분', 0.649),
 ('오래간다', 0.646),
 ('강쇠', 0.6445),
 ('ISP', 0.6436),
 ('나사', 0.638),
 ('여분', 0.637),
 ('타폰', 0.637),
 ('껍데기', 0.6367),
 ('가로길이', 0.636),
 ('기판', 0.6353),
 ('단다고', 0.634),
 ('전지', 0.6333),
 ('암페어', 0.6333),
 ('배불뚝이', 0.6323),
 ('극심', 0.6323),
 ('뽀대', 0.6313),
 ('상판', 0.6304),
 ('회로', 0.6294),
 ('접점', 0.6294),
 ('RMAA', 0.629),
 ('온도', 0.6284),
 ('외장형', 0.6284),
 ('mAH', 0.628),
 ('런타임', 0.6274),
 ('ap', 0.625),
 ('본체', 0.625),
 ('코랄', 0.625),
 ('아이피', 0.6235),
 ('통했', 0.6235),
 ('옹이', 0.622),
 ('넓이', 0.6216),
 ('절벽', 0.6216),
 ('밀리암페어', 0.6216),
 ('Ips', 0.621),
 ('tta', 0.621),
 ('Ap', 0.62),
 ('주범', 0.62),
 ('ram', 0.6196),
 ('늘어납니다', 0.619),
 ('늘려서', 0.618),
 ('하판', 0.618),
 ('풀로', 0.617),
 ('안드

## Document-Term Matrix 구축

In [45]:
# 스마트하게 일일히 lookup 하지 않고 document-term matrix로 구축하기 
from sklearn.feature_extraction.text import CountVectorizer #dtm을 빠르게 해주는 라이브러리 
vectorizer = CountVectorizer(vocabulary=vocab, #vocab에 해당하는 단어들만 사용한다고 강제하는 파라미터, 지금 데이터가 너무 크니까 vocab을 줄이는 방법 추전
                             binary=True, # 2번 쓰였어도 1이라고 세라는 파라미터
                             ngram_range=(1, 1), #한 단어씩 슬라이딩해서 써라 
                             tokenizer=lambda x: x.strip().split()) #토크나이저가 안 되어 있어서 한 문장 x를 공백으로 split한걸 토크나이저 함수로 써라
corpus = open("data/phone_review_mecab.txt", "r", encoding="utf-8").readlines()
corpus = list(set([el for el in corpus if len(el.split()) < 31])) #.split(줄 바꿈 문자 없애기)
dtm = vectorizer.fit_transform(corpus).toarray() #corpus를 넣고 fit_transform을 하면 dtm이 완성됨. 
word_counts_per_doc = np.sum(dtm, axis=1) # 길이가 길어질수록 score값이 높아지는 걸 방지하기 위해 길이로 나눔(axis=1()
result = np.dot(dtm, weight_matrix.T) # dtm 형성하는 데는 오래 걸리지 않는데, dot product가 오래 걸림. (0.9 + 0.1 ---- 이런 값 만드는 게 오래 걸림 (ppt 참고) ))

In [27]:
corpus

['ㄷ ㄷ ㄷ 디 자 이 어 HD 나 HD 2 같이 액정 큰 녀석 들 이 떙기던데 ㅎㅎ\n',
 '겔 넥 진짜 사 고 싶 다 ㅋㅋ ㅋㅋ 으아 ㅋㅋ\n',
 '전 갤 스\n',
 '흠 엠 넷 무제한 모바일 티비 무제 한 마음 에 드 네요 근데 헬 지 모바일 티비 화질 좀 어 케 해 주 지 @ @\n',
 'kt 갤럭시 s 2 예약 하 신 분\n',
 '2 0 만 원 정도 를 사정 권 에 두 고 있 긴 합니다\n',
 '나름 아이언맨 포터블 브리프 케이스 버전 삘도 나 고 그러 네요 ㅎ\n',
 '옵 이 이 블랙 소유 하 고 있 어요 ㅋㅋ 테이크 와서 사용 해 봤 구요\n',
 '갤럭시 넥서스 도 나오 고 갤 투 할 원 이나 좀 떨 어 졌으면 노트 는 저 한테 너무 애매\n',
 '기자 들 이 죄다 \\ 향 \\ 을 쓰 네요 에휴 걍 해 외용 이 라고 써도 되 지 않 나요\n',
 'Xp 에서 는 연결 안 되 도 윈 7 에서 는 안 된 적 이 한 번 도 없 는데 안 되 기 도 하나 보 네요\n',
 '갤 2 에 젤리빈 먹이 면\n',
 '시리우스 단점 혹은 불편 한 점\n',
 '블루 블랙 블루 블랙\n',
 '저 도 스 크 에서 넘어가 야 되 는데 헬 지 는 부 가 서비스 가 덕지덕지 라 개티 만 기다리 고 있 습니다 ㅜㅜ\n',
 '음 좋 은 하루 시작 하 는 좋 은 글\n',
 '그냥 제 값 다 주 고 사신 거 같 네요 눈 탱이 는 아니 라 다 ㅋ 행\n',
 '영상 이 너무 커요 ㅠㅠ 좀 만 더 축소 해 줬 으면\n',
 'ㅠㅠ 저희 도 용\n',
 '노트 1 6 기 가 화이트 가 3 0 만 원 대 들어온다면 ㅠ 블랙 3 9 에 질러 버린 사람 으로써 개 철 해야 되 나요 ㅋㅋ\n',
 '네 안 됩니다 ㅠ ㅠ 외산 폰 은 거의 통화 녹음 을 지원 하 지 않 습니다\n',
 '명 의 이전 이 되 면 좋 겠 는데 안 된다고 하 네요\n',
 '국내 기사 면 애플 프리미엄 의 이미지 포기 인가 이제 는 애플 이 삼성 따라 하 기 인가 뭐 이런 기사 가\n',
 

In [31]:
dtm # 일부 어휘만 쓰이기 때문에 0이 많음

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
dtm.shape # 문서 사이즈(말뭉치 크기 courpus, dtm 개수) 

(203606, 23599)

In [37]:
weight_matrix.shape #이거랑 dtm이랑 내적한 게 우리가 원하는 battery score가 나옴. 

(6, 23599)

In [34]:
dtm[0]

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
word_counts_per_doc # 첫번재 문장에 단어 13, 두번째엔 9개, 3번째엔 3개....

array([13,  9,  3, ...,  6, 20,  6])

In [29]:
word_counts_per_doc ** 0.6 # 13으로 나눠지면 너무 짧은 문장만 나오니까 대충 0.6 제곱해주면 되지 않을까? -> 그러면 긴 문장도 나옴. 

array([4.65978642, 3.73719282, 1.93318204, ..., 2.93015605, 6.03417634,
       2.93015605])

## 최종 결과물 도출

In [40]:
a = np.array([0.1,0.2,0.3])

In [42]:
a.argsort() # 높은 순으로 정렬하고 싶을 때 

array([0, 1, 2])

In [43]:
a = - np.array([0.1,0.2,0.3])

In [44]:
a.argsort() # - 넣으면 높은순으로 정렬이 됨. 그래서 밑에도 - result로 해준거임. 

array([2, 1, 0])

In [38]:
for func_idx, func in enumerate(functions):
    print("##### " + func) #result( 기능 스코어 x dtm 내적한 값 => result = np.dot(dtm, weight_matrix.T)
    scores = - result[:, func_idx] / (word_counts_per_doc ** 0.6) # - result 0.0 행 벡터 값을 높은 순서로 나오게 해주고(배터리 스코어)/ 그걸 글자 단어들로 나누어줌
    # 13으로 나눠지면 너무 짧은 문장만 나오니까 대충 0.6 제곱해주면 되지 않을까? -> 그러면 긴 문장도 나옴. 
    for sent_idx in scores.argsort()[:10]: #큰 순으로 정렬되지만 10개만 나오게 하게 
        print(corpus[sent_idx].strip(), - scores[sent_idx]) #argsort는 인덱스만 나옴, 그니까 내용과 스코어를 뽑아오게 프린트를 만들어줌 
    print("\n")
    # 각 관심 스코어에 대해 가중치가 높은 문장들이 쫘르륵 나옴

##### 배터리
헐 저 도 그런데 완충 상태 표준형 끼워 놓 고 자 고 일어났 더니 5 0 가 줄 어 있 더라구요 근데 또 대용량 은 멀쩡 한데 배터리 문제 인가요 1.7018879721141027
X 6 하나 면 고성능 내장 라디오 Mp 3 카메라 통 품 1 6 G 메모리 변강 쇠 배터리 까지 3 5 인치 면 정말 좋 았 을 텐데 ㅠㅠ 1.6796281093383703
제 갤 s 3 배터리 두 개 중 하나 는 부풀 었 고 부푼 녀석 은 광탈 입니다 1 년 다 되 어 가 서 교환 도 안 되 죠 1.673625151827325
시리우스 그 두 개 외 에 도 배터리 표준형 슬림 이랑 대용량 있 어서 커버 2 개 가지 고 다녀야 되 는 게 너무 압박 이 네요 ㅎ ㄷ ㄷ 1.6722951119995961
그 럴리가요 저 는 미라크 유일 한 불만 이 배터리 인데 ㅎㅎ 아 그리고 윗 님 금오 충전기 통합 2 0 핀 으루 충전 하 심 꽤 빨라요 1.6681817313439171
결국 노트 2 에 와서야 배터리 가 완전체 가 된 것 같 군요 노트 1 갤 삼 은 배터리 광탈 도 모자라 서 스웰 링 현상 까지 1.6510252745858631
아몰 레기 가 배터리 효율 더 좋 지 않 나요 밝기 는 넘사벽 인데 갤 삼 밧데리 정말 오래 갑니다 IPS 나오 면 단점 도 있 어요 무거워 집니다 1.6460421931969305
갤 치디 는 스냅 3 세대 AP 를 사용 했으며 성능 과 배터리 면 에서 모두 엑시노스 4 2 1 0 에 밀립 니 다 1.6449100960810201
이 제품 은 배터리 커버 가 있 기 떄문에 주의 를 해야 합니다 실링 에 이 물질 이 끼 거나 덜 닫힌 상태 에서 담그 면 사망 하 는 겁니다 1.6346839321007227
5 3 인치 배터리 교체 가능 아몰 래드 Q 보이스 미 지원 CPU 스냅 3 세대 램 1 기 가 음 그냥 갤럭시 S 2 HD 가 화면 만 커진 1.62872760424487


##### 촬영
인터넷 사진 촬영 동영상 촬영 SNS 사용

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
for func_idx, func in enumerate(functions):
    print("##### " + func)
    scores = - result[:, func_idx] / (word_counts_per_doc ** 0.6)
    i = 0
    for sent_idx in scores.argsort():
        if func not in corpus[sent_idx] and i < 10:
            print(corpus[sent_idx].strip(), - scores[sent_idx])
            i += 1
    print("\n")

##### 배터리
미친 듯 한 발열 3 G 가끔 끊김 박 대리 조기 퇴근 충전 속 도 드럽 게 느림 유격 홈 버튼 빛 샘 카메라 뻘건 멍 등등 1.617438035081736
분명 아트 가 낳 지만 국산 어 플 구동력 이 딸리 죠 아크 는 오줌 액정 베가스 는 정체 가 뭐 지 베레기 답 은 뭐 다 갤 하 트 1.6158825403053065
내장 메모리 를 늘려서 스냅드래곤 을 카바 치 는 건가 밧데리 2 개 랑 내장 메모리 용량 정도 가 해외 판과 의 경쟁력 은 아니 겠 죠 1.6116792316184538
전 헬레 이 서 인대 밧데리 가 조루 인 거 빼 건 성능 은 쓸 만 합니다 뒤 껍데기 2 개 나 주 고 액 보 까지 ㅋ 1.610589797987608
거치대 가 충전 도 되 지만 HDMI 출력 을 2 만 원 짜리 프라다 MHL 커넥터 ㄷ ㄷ 저 R 3 쓰 는데 호환 만 된다면 사 고 싶 은데요 1.6064760527607995
제조 업체 3 사 의 하이엔드 G 2 갤 4 ltea 베티 아 베 시업 스펙 은 동일 합니다 디자인 이나 액정 이나 ui 측면 에서 호불호 가 갈릴 뿐 1.6023368010008827
테티 이랑 베가스 랑 같이 쓰 는 중 인데 인터넷 이랑 속도 는 발군 메모리 랑 어 플 저장 공간 은 어휴 ㅡㅡ 베 터리 도 오래가 고 1.5959308453715508
카메라 는 알파 부터 확실히 우위 였 죠 센서 크롭 ois 의 부제 만 빼 면 노트 4 와 거의 같 은 스펙 에 품질 도 1.592688384861217
전 SGP 가 안 나와서 기다리 다 걍 트라이 디아 에어 쟈 켓 노란색 사 서 가지 고 댕기 는데 완전 이쁨 ㅋ 참고 로 남자 임 ㅎㅎㅎ 1.5909032730446075
제 가 쓰 고 있 는데 롤리팝 떄나 마시멜로 우 떄나 거의 완전 체급 입니다 4 3 비 와 충전 속도 뺴고는 단점 이 없 는 기계 입니다 1.5901977249850379


##### 촬영
동영상 자주 보 시 면 옵 lte 가 좋 아요 정말 재 생력 발군 입니

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until




##### 디자인
아니 애초 에 둘 이 색깔 자체 가 다른데요 삼성 꺼 는 금괴 에 가까운 진한 골드색 이 고 아이폰 은 연한 샴페인 골드색 인데 단순히 골 드라고 1.6736747926559343
다시 와이드 비율 로 돌아왔 고 해상도 가 올라갔 네요 근데 스테레오 스피커 한쪽 에 몰려 있 는 건 진짜 멍청 하 고 쓸모없 는 구조 같 은데 대체 왜 1.6025084655719608
걸립니다 이번 판결 은 외형 보다 ux 에 초점 이 라 핀투 터치 이런 건 얄짤없 죠 갤 삼 뿐 만 아니 라 모든 안 드 폰 은 다 걸립니다 1.5786197987721788
카메라 는 알파 부터 확실히 우위 였 죠 센서 크롭 ois 의 부제 만 빼 면 노트 4 와 거의 같 은 스펙 에 품질 도 1.5758890412757809
어우 여긴 전쟁터 네요 ㅋ 확실 한 건 LG 가 많이 따라왔 고 삼성 과 거의 비등 하 다는 거 죠 말 들 을 그렇게 어렵 게 하 는 건지 1.5729408779534668
분명 아트 가 낳 지만 국산 어 플 구동력 이 딸리 죠 아크 는 오줌 액정 베가스 는 정체 가 뭐 지 베레기 답 은 뭐 다 갤 하 트 1.5687337653042062
제 상전 도 아니 고 한낱 물건 따위 를 부르 는데 무슨 용기 씩 이나 아무리 뭐 라고 해도 화이트 노이즈 등 음감 성능 떨어지 는 건 사실 이 죠 1.565689020285382
아크 퍼포먼스 는 갤 스랑 비교 하 기 도 감지덕지 죠 실제로 더 떨어지 구요 더구나 발적 화 괜히 일본 에서 조차 쿠소 페리아 라고 불리 는 게 아니 니 1.5621278401805188
갠 적 으로 노란색 보라색 파랑 색 이런 원색 좀 나오 지 옛 날 에 롤리팝 이나 쿠키 폰 이 색깔 은 굿 이 었 는데 ㅋㅋ 1.5552327685537115
블랙 이 괜츈한듯 난 화이트 신봉자 인데 미라크 화이트 는 싼 티 가 좔좔 초딩 폰 옆 에 이상 한 크롬 같 은 거 다 뜯어내 고 싶 음 1.5429471185660677


##### 화면
다시 